# Подход через скользящее среднее по логарифмированному значению продаж

In [6]:
import numpy as np
import pandas as pd
from datetime import timedelta

In [21]:
# Загрузим данные. Будем использовать не весь датасет, а его часть
train = pd.read_csv('./data/train.csv', usecols=[1,2,3,4],
                    dtype={'id':'uint32', 'item_nbr':'int32', 'store_nbr':'int8', 'unit_sales':'float32'}, parse_dates=['date'],
                    skiprows=range(1, 106672217)
                    )
test = train.loc[train.shape[0]-10000000:,:]
train = train.loc[:train.shape[0]-10000000,:]
train.head()

,date,store_nbr,item_nbr,unit_sales
0,2017-02-18,34,463901,1.0
1,2017-02-18,34,463903,2.0
2,2017-02-18,34,464263,2.0
3,2017-02-18,34,464302,3.0
4,2017-02-18,34,464333,26.0


In [22]:
# Итак, посчитаем скользящее среднее по дням недели
train.loc[(train.unit_sales < 0), 'unit_sales'] = 0 
train['unit_sales'] = train['unit_sales'].apply(np.log1p)
train['dow'] = train['date'].dt.dayofweek

# На основе tarobxl: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/discussion/42948
ma_dw = train.groupby(['item_nbr', 'store_nbr', 'dow'])['unit_sales'].mean().reset_index()
ma_dw.rename(columns={'unit_sales': 'madw'}, inplace=True)
ma_wk = ma_dw.groupby(['item_nbr', 'store_nbr'])['madw'].mean().to_frame('mawk').reset_index()

train.drop('dow', axis=1, inplace=True)

train.head()


,date,store_nbr,item_nbr,unit_sales
0,2017-02-18,34,463901,0.693147
1,2017-02-18,34,463903,1.098612
2,2017-02-18,34,464263,1.098612
3,2017-02-18,34,464302,1.386294
4,2017-02-18,34,464333,3.295837


In [23]:
ma_dw.head()

,item_nbr,store_nbr,dow,madw
0,96995,1,0,0.693147
1,96995,1,1,0.693147
2,96995,1,2,0.693147
3,96995,1,4,0.895880
4,96995,1,5,1.386294


In [24]:
ma_wk.head()

,item_nbr,store_nbr,mawk
0,96995,1,0.872323
1,96995,2,0.876249
2,96995,3,0.803386
3,96995,6,0.874713
4,96995,7,0.767574


Что мы с вами сделали: посчитали скользящее среднее от логарифма по дням недели. Почему от логарифма? чтобы сглаживание работало лучше, смотрите: если у вас есть яблоки и они продаются по 10000 за день, то ошибиться на 10 яблок не так страшно, если у вас дорогая премиум ветчина, которая продается 5 штук за день, то ошибка на 10 штук будет критичной. Логарифм позволяет сгладить это.

In [25]:
# Препроцессинг
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
).reset_index()

del u_dates, u_stores, u_items

train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs
lastdate = train.iloc[train.shape[0]-1].date

/var/folders/df/dz8wtw0j7791hg98mrx_thxc0000gn/T/ipykernel_36676/3389827123.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train.loc[:, 'unit_sales'].fillna(0, inplace=True) # fill NaNs


In [32]:
#Moving Averages
ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(
        ['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais')

for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')

del tmp,tmpg,train

ma_is['mais']=ma_is.median(axis=1)
ma_is.reset_index(inplace=True)
ma_is.drop(list(ma_is.columns.values)[3:],axis=1,inplace=True)

NameError: name 'train' is not defined

In [30]:
test.head()

,date,store_nbr,item_nbr,unit_sales,dow,mais,mawk,madw,unit_sales_predict
0,2017-05-13,22,1091368,5.0,5,1.148429,1.390658,1.723364,1.423182
1,2017-05-13,22,1091369,2.0,5,1.165779,1.381940,1.601625,1.351101
2,2017-05-13,22,1094238,3.0,5,0.458053,0.929120,1.033206,0.509367
3,2017-05-13,22,1096235,1.0,5,1.465946,1.794866,2.085608,1.703407
4,2017-05-13,22,1098624,6.0,5,2.112757,2.165380,2.307599,2.251520


In [31]:
# Применим модель
test['dow'] = test['date'].dt.dayofweek
test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

del ma_is, ma_wk, ma_dw

#Forecasting Test
test['unit_sales_predict'] = test.mais
pos_idx = test['mawk'] > 0
test_pos = test.loc[pos_idx]
# Посчитаем предсказание от скользящего среднего, при этом не забываем экспоненциировать
test.loc[pos_idx, 'unit_sales_predict'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales_predict"].fillna(0, inplace=True)
test['unit_sales_predict'] = test['unit_sales_predict'].apply(np.expm1)

NameError: name 'ma_is' is not defined

In [10]:
# Посчитайте полученное значение ошибки на тестовом срезе
